In [1]:
import sys
sys.path.append('../jupyterlab_sparksql/')

In [2]:
from pyspark.sql import SparkSession
spark = SparkSession.builder.master("local[*]").getOrCreate()

%load_ext magics
%load_ext autoreload
%autoreload 2

<IPython.core.display.Javascript object>

In [3]:
table2 = spark.read.csv('people-2000000.csv', header=True, inferSchema=True).drop('Index')

In [64]:
%%sql xd 
select
    *
from (values
    (0,null, '2023-05-31', 'line1\nline2'),
    (2,0.24, '2023-04-01', 'line3\nline2'),
    (1,0.56, '2023-05-01', 'line2\nline2'),
    (7,0.99, '2023-03-01', 'line5\nline2'),
    (10,float('-inf'), '2023-03-01', 'line4\nline2')
      
) as t(id, scale_score, update_time, test)
order by scale_score desc

Output()

In [8]:
table1.toPandas().dtypes

id              int32
scale_score    object
update_time    object
test           object
dtype: object

In [77]:
%%sql -n 1000
select
    *
from table2

Output()

In [22]:
table2.toPandas().dtypes

User Id          object
First Name       object
Last Name        object
Sex              object
Email            object
Phone            object
Date of birth    object
Job Title        object
dtype: object

In [114]:
import pandas as pd
import json

def map_dtypes(df):
    # Define the mapping of Pandas data types to JavaScript data types
    dtype_map = {
        'int64': 'Number',
        'float64': 'Number',
        'object': 'String',
        'bool': 'Boolean',
        'datetime64[ns]': 'Date'
    }
    # Create an empty list to store the JavaScript data types of the columns
    col_types = []
    # Map the data types of the columns in the DataFrame
    for col in df.columns:
        if str(df[col].dtype) in dtype_map:
            col_types.append(dtype_map[str(df[col].dtype)])
    # Convert the col_types list to a JSON string and return it
    return json.dumps(col_types)

# Example usage
df = pd.DataFrame({
    'A': [1, 2, 3],
    'B': [1.1, 2.2, 3.3],
    'C': ['a', 'b', 'c'],
    'D': [True, False, True],
    'E': pd.date_range('2022-01-01', periods=3)
})
col_types_json = map_dtypes(df)
print(col_types_json)


["Number", "Number", "String", "Boolean", "Date"]


In [29]:
%%sql table3
select
    `User Id` as user_id
    , `First Name` as first_name
    , `Last Name` as last_name
    , Sex as sex
    , Email as email
    , Phone as phone
    , `Date of birth` as dob
    , `Job Title` as job_title
from table2

%%sql
select
    *
    , regexp_replace(job_title, r'\s', '\n') as job_title2
from table3

In [11]:
import ipywidgets as w

In [12]:
w.Text(
    placeholder='Search',
    disabled=False ,
    icon='search',
    layout=w.Layout(
        width='150px',
        margin='1px 10px 2px 2px'
    )
)

Text(value='', layout=Layout(margin='1px 10px 2px 2px', width='150px'), placeholder='Search')

In [13]:
ipd.display(ipd.HTML(
"""
<div class="lm-Widget p-Widget jupyter-widgets widget-inline-hbox widget-text" style="margin: 1px 10px 2px 2px; width: 150px;">
    <input type="text" id="" placeholder="Search">
</div>
"""
))

NameError: name 'ipd' is not defined

In [ ]:
from IPython import display as ipd

In [ ]:
html_string = """
<table id="T_0a393" style="border-collapse:separate">
  <thead>
    <tr>
      <th class="blank level0">&nbsp;</th>
      <th id="T_0a394_level0_col0" class="col_heading level0 col0">user_id</th>
</table>
"""

In [ ]:
import re

In [ ]:
match = re.search(r'<table.*?id="(.*?)".*?>', html_string)
if match:
    table_id = match.group(1)
    print(table_id)

In [ ]:
def extract_table_id(html_table):
    match = re.search(r'<table.*?id="(.*?)".*?>', html_table)
    if match:
        return match.group(1)

In [ ]:
extract_table_id(html_string)

In [47]:
import pandas as pd
df = pd.DataFrame({"col": ["a", "b", "c"]})
df.style.relabel_index(["A", "B", "C"]).hide([0,1])


,col
C,c


In [48]:
# hide first, then relabel
df = pd.DataFrame({"col": ["a", "b", "c"]})
df.style.hide([0,1]).relabel_index(["C"])

,col
C,c


In [58]:
import re

table_html = """<thead>
    <tr>
      <th class="blank level0">#</th>
      <th id="T_dfd8a_level0_col0" class="col_heading level0 col0">id</th>
      <th id="T_dfd8a_level0_col1" class="col_heading level0 col1">scale_score</th>
      <th id="T_dfd8a_level0_col2" class="col_heading level0 col2">update_time</th>
      <th id="T_dfd8a_level0_col3" class="col_heading level0 col3">test</th>
    </tr>
  </thead>"""

thead = re.search(r'<thead>[\s\S]*?</thead>', table_html)
if thead:
    # Extract the content of the thead tag
    thead_content = thead.group()
    # Find all th tags within the thead tag
    th_tags = re.findall(r'<th.*?>.*?</th>', thead_content)
for i, tag in enumerate(th_tags):
    tag_open = re.findall(r'<th[^e][^>]*>', tag)[0]
    tag_content = re.findall(r'(?=>)(.*)(?=</th>)', tag)[0] if i != 0 else '#'
    new_tag = tag_open[:-1] + f' onclick="(function(){{ sortTable(\'T_dfd8a\', {i}) }})()">' + tag_content + '<i class="fa fa-sort" aria-hidden="true"></i>'
    table_html = table_html.replace(tag, new_tag)
print(table_html)

<thead>
    <tr>
      <th class="blank level0" onclick="(function(){ sortTable('T_dfd8a', 0) })()">#<i class="fa fa-sort" aria-hidden="true"></i>
      <th id="T_dfd8a_level0_col0" class="col_heading level0 col0" onclick="(function(){ sortTable('T_dfd8a', 1) })()">>id<i class="fa fa-sort" aria-hidden="true"></i>
      <th id="T_dfd8a_level0_col1" class="col_heading level0 col1" onclick="(function(){ sortTable('T_dfd8a', 2) })()">>scale_score<i class="fa fa-sort" aria-hidden="true"></i>
      <th id="T_dfd8a_level0_col2" class="col_heading level0 col2" onclick="(function(){ sortTable('T_dfd8a', 3) })()">>update_time<i class="fa fa-sort" aria-hidden="true"></i>
      <th id="T_dfd8a_level0_col3" class="col_heading level0 col3" onclick="(function(){ sortTable('T_dfd8a', 4) })()">>test<i class="fa fa-sort" aria-hidden="true"></i>
    </tr>
  </thead>


In [50]:
import re

text = """
<tr>
      <th class="blank level0"> </th>
    </tr>
"""

pattern = r'<th.*?>(.*?)</th>'
matches = re.findall(pattern, text)

for match in matches:
    print(match)
